Sources:

1) virtual-youtuber.userlocal.jp - Website that stores a massive list of existing VTubers

2) youtube.com - world famous video hosting

3) twitch.tv - world famous streaming service

4) playboard.co - collectoin of the data for YouTube channels worldwide to determine and announce reliable rankings

In [354]:
from selenium import webdriver
from selenium.common import exceptions
import warnings
import time
import pandas as pd
import csv

warnings.filterwarnings('ignore')

## Part 1


I want to build a graph of time and number of appearing vtubers. I will get needed data by using selenium. Data about existing Vtubers is obtained from virtual-youtuber.userlocal.jp

In [74]:
#web-site url assignment
url = 'https://virtual-youtuber.userlocal.jp/document/ranking?page='

#creating puppet browser
browser = webdriver.Firefox()

#create dictionary of lists to write down parsed vtuber names and subscribers' count
vtuber = {}
vtuber['name'] = []
vtuber['subs'] = []
vtuber['registration date'] = []

#list of vtubers has 40 pages, so go through them and concatenate page number with site's url
for i in range(1, 41):
    browser.get(url + str(i))
    time.sleep(2)#wait 2 seconds to let web-site load
    
    #try-except block that finds html elements that I need
    try:
        name = browser.find_elements_by_class_name('no-propagation')
        subs = browser.find_elements_by_class_name('text-success')
        name = name[1::3]#to delete unneeded elements
    
    except:
        print('An Exception has occurred.')
    
    #assign obtained values to appropriate lists of dictionary
    for i in range(len(name)):
        vtuber['name'].append(name[i].text)
        vtuber['subs'].append(subs[i].text)

#close browser
browser.quit()

Now let's obtain data about registration date of each VTuber in the set. Luckily for us, youtube.com does provide desired information

In [279]:
#Scraping took about 10 hours due to amount of query and delay between them
#Create puppet browser and make it access youtube.com
browser = webdriver.Firefox()
browser.get('https://www.youtube.com')


#go through every vtuber in the set
for i in range(len(vtuber['name'])):
    
    #let web-site load its objects
    time.sleep(3)
    
    try:
        #find searchbox, write down vtuber name and add 'ch' to be sure that first link to output will be desired channel
        searchbox = browser.find_element_by_tag_name('input')
        searchbox.send_keys(vtuber['name'][i])
        searchbox.submit()
        
        #Let the page download, find filter button and filter outputs so that they show only channels
        time.sleep(6)
        filter_dropdown = browser.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a')
        filter_dropdown.click()
        filter_channels = browser.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[2]/ytd-search-filter-renderer[2]/a/div/yt-formatted-string')
        filter_channels.click()
        
        #wait for 2 seconds to let the page load
        time.sleep(2)
        #find channel link and click it
        channel_link = browser.find_element_by_xpath('//*[@id="main-link"]')
        channel_link.click()
        
        #wait for a while to let page load, go to channel description and get registration data
        time.sleep(2)
        channel_description = browser.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-browse/div[3]/ytd-c4-tabbed-header-renderer/tp-yt-app-header-layout/div/tp-yt-app-header/div[2]/tp-yt-app-toolbar/div/div/tp-yt-paper-tabs/div/div/tp-yt-paper-tab[6]/div')
        channel_description.click()
        
        
        registr_date = browser.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-channel-about-metadata-renderer/div[2]/yt-formatted-string[2]/span[2]')
        
        #Clear searchbox to start again
        browser.find_element_by_tag_name('input').send_keys(Keys.CONTROL, 'a')
        browser.find_element_by_tag_name('input').send_keys(Keys.DELETE)
        
    #handle exceptions by skipping null elements    
    except (exceptions.StaleElementReferenceException, exceptions.NoSuchElementException) as e:
        print(e)
        browser.find_element_by_tag_name('input').send_keys(Keys.CONTROL, 'a')
        browser.find_element_by_tag_name('input').send_keys(Keys.DELETE)
        vtuber['registration date'].append('null')
            
        continue
    
    except (exceptions.WebDriverException, exceptions.ElementNotInteractableException) as e:
        print(e)
        browser.find_element_by_tag_name('input').send_keys(Keys.CONTROL, 'a')
        browser.find_element_by_tag_name('input').send_keys(Keys.DELETE)
        vtuber['registration date'].append('null')
        continue
        
    #add it to our dictionary of lists
    vtuber['registration date'].append(registr_date.text)

browser.quit()

In [391]:
# Formate dates: split them by quarters of years

q1 = ['янв', 'фев', 'мар']
q2 = ['апр', 'мая', 'июн']
q3 = ['июл', 'авг', 'сен']
q4 = ['окт', 'ноя', 'дек']

for i in range(len(vtuber['registration date'])):
    for j in range(3):
        if q1[j] in vtuber['registration date'][i]:
            new_date = vtuber['registration date'][i][:-3].split()
            new_date[0:2] = 'Q1, '
            vtuber['registration date'][i] = ''.join(new_date)
            
        elif q2[j] in vtuber['registration date'][i]:
            new_date = vtuber['registration date'][i][:-3].split()
            new_date[0:2] = 'Q2, '
            vtuber['registration date'][i] = ''.join(new_date)
            
        elif q3[j] in vtuber['registration date'][i]:
            new_date = vtuber['registration date'][i][:-3].split()
            new_date[0:2] = 'Q3, '
            vtuber['registration date'][i] = ''.join(new_date)
            
        elif q4[j] in vtuber['registration date'][i]:
            new_date = vtuber['registration date'][i][:-3].split()
            new_date[0:2] = 'Q4, '
            vtuber['registration date'][i] = ''.join(new_date)

In [392]:
#Now write obtained data to the csv file
fieldnames = ['name', 'subs', 'registration date']

with open('Datasets/vtubers_date.csv', 'w', encoding = 'utf-8') as f:
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    for i in range(len(vtuber['name'])):
        w.writerow({'name': vtuber['name'][i], 'subs': vtuber['subs'][i], 'registration date': vtuber['registration date'][i]})
f.close()

## Part 2

Dataset from kaggle contains only data from youtube, but VShojo agency actually is a twitch based agency and that's why their members' stats are different from the truth. Here I am going to fix it and obtain info about them from twitch.tv

In [33]:
#Take only those rows from dataframe, whose affiliation is VShojo
df = pd.read_csv('Datasets/channels.csv')
df = df[df.affiliation == 'VShojo']

#Sort it by deleting archive channels
vshojo_names = list(df[~df.name.str.contains('Archive')]['name'])
vshojo_names = [x for x in vshojo_names if 'VOD' not in x]
vshojo_names = [x for x in vshojo_names if 'VShojo' not in x]
vshojo_names[-2] = 'Apricot'

vshojo_names

['Projekt Melody',
 'ironmouse',
 'Silvervale',
 'Nyanners',
 'HimeHajime',
 'Veibae',
 'Apricot',
 'Zentreya']

In [393]:
#Create dictionaries. One to store VShojo members' data and another to store html objects' addresses
vshojo = {}
vshojo['name'] = []
vshojo['subs'] = []

html_classes = {'searchbox': 'btlhBn', 'button': 'fSetzA', 'link': 'fcQsXy',
           'xpath': '/html/body/div[1]/div/div[2]/div[1]/main/div[2]/div[3]/div/div/div[1]/div[1]/div[2]/div/div[2]/div[1]/div[1]/div[2]/p'}

#Create puppet browser, open twitch and wait for 5 seconds to let it load
browser = webdriver.Firefox()
browser.get('https://www.twitch.tv')
time.sleep(5)

#go through every VShojo member
for name in vshojo_names:
    try:
        #Find Searchbox, write down member's name and click find button to activate search engine
        searchbox = browser.find_element_by_class_name(html_classes['searchbox'])
        button = browser.find_element_by_class_name(html_classes['button'])
        searchbox.send_keys(name)
        button.click()
        
        #let a new page load and find link, leading to channel description
        time.sleep(2)
        link = browser.find_elements_by_class_name(html_classes['link'])
        link[0].click()
        
        #let a new page load and finally obtain channel information
        time.sleep(5)
        follower_count = browser.find_element_by_xpath(html_classes['xpath'])        
    except:
        print('Was not able to find an element with that name.')

    #add data to dictionary
    vshojo['name'].append(name)
    vshojo['subs'].append(follower_count.text)

#close browser and output obtained data(just for curiosity)
browser.quit()
print(vshojo)

{'name': ['Projekt Melody', 'ironmouse', 'Silvervale', 'Nyanners', 'HimeHajime', 'Veibae', 'Apricot', 'Zentreya'], 'subs': ['512,5 тыс. фолловеров', '825,1 тыс. фолловеров', '237,8 тыс. фолловеров', '690,8 тыс. фолловера', '103,4 тыс. фолловеров', '626,4 тыс. фолловера', '272,2 тыс. фолловеров', '247,5 тыс. фолловеров']}


In [394]:
#convert data to a csv file, so that we could use it in our project
fieldnames = ['name', 'subs']

with open('Datasets/vshojo.csv', 'w', encoding='utf-8') as f:
    w = csv.DictWriter(f, fieldnames = fieldnames)
    w.writeheader()
    for i in range(len(vshojo['name'])):
        w.writerow({'name': vshojo['name'][i], 'subs': vshojo['subs'][i]})
f.close()

## Part 3

I want to get a tiny amount of data just to show the most profitable channels ever

In [347]:
top = {}
top['name'] = []
top['revenue'] = []

browser = webdriver.Firefox()
browser.get('https://playboard.co/en/youtube-ranking/most-superchatted-all-channels-in-worldwide-total')
try:
    revenue = browser.find_elements_by_class_name('score')
    name = browser.find_elements_by_tag_name('h3')
except:
    print('An exception has occured')

revenue = revenue[2::4]
name = name[5:]
for i in range(len(revenue)):
    top['name'].append(name[i].text)
    top['revenue'].append(revenue[i].text)
    
browser.quit()

In [353]:
fieldnames = ['name', 'revenue']

with open('Datasets/top_revenue.csv', 'w', encoding = 'utf-8') as f:
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    for i in range(len(top['name'])):
        w.writerow({'name': top['name'][i], 'revenue': top['revenue'][i]})
f.close()